# Amazon Lookout for Equipment no-code Workshop
---
No-code workshop to experiment with Amazon Lookout for Equipment using only the AWS console.

To get started with Lookout for Equipment, you can do the following:
* Create a project and ingest some historical data
* Train an anomaly detection model using part of the ingested data
* Evaluate the model on a left-out part of the ingested data
* Configure and launch an inference scheduler

## Region selection
Please, check with your instructor about the regions where you will be working for this hands-on workshop. You can change it by selecting the region name in the upper right of the screen:

![Region selection](pictures/region_selection.png)

At this time, Lookout for Equipment is available in Europe (Ireland, **eu-west-1**), US East (N. Virginia, **us-east-1**) and Asia Pacific (Seoul, **ap-northeast-2**).

## Creating an S3 Bucket
Lookout for Equipment imports the training data from any S3 bucket. Let's create one to store the files we will need during this workshop.

1. Access the S3 management console by typing **S3** in the search bar as shown below and click on S3 to open the service console.

![S3 console access](pictures/s3_service_search.png)

2. Click on **Create bucket**
3. Enter a **Bucket name**:  for the remaining of this document, we will reference a bucket named *lookout-equipment-no-code-workshop* (as bucket names need to be unique across the S3 service, yours will have to be named otherwise). Scroll down and click on **Create bucket**
4. You are brought back to the bucket list from your account. Click on the name of your bucket and create the following folders:
    * `facility/`
    * `label-data/`
    * `inference-data/input/`
    * `inference-data/output/`
    
Your bucket content should look similar to the following:

![S3 Bucket initial content](pictures/s3_bucket_root.png)

Your bucket is ready to receive some data, let's download and prepare them.

## Prepare input data
### Download data for learning
In this hands-on exercise, you will use a synthetic dataset where all sensors are located in one file (there are 30 of them). Download training and label data to your computer using the following links:
* Click on [**this link**](https://lookoutforequipmentbucket-us-east-1.s3.amazonaws.com/datasets/getting-started/console/pump_sensors.zip) to download the `pump_sensors.zip` archive and unzip it. You end up with a 250 MB time series file named `pump_sensors.csv`.
* Click on [**this link**](https://lookoutforequipmentbucket-us-east-1.s3.amazonaws.com/datasets/getting-started/console/labels.csv) to download the `labels.csv` file

For this tutorial we will use this synthetic dataset and only use a version where all sensors are in one single CSV file. The first column of this file is a timestamp, so this file contains 31 columns and 432,000 rows. The following are sample rows from this time series dataset:

![Synthetic pump data timeseries overview](pictures/pump_data_overview.png)

We have access to 10 months of data with a regular sampling rate of 1 minute for every sensors.

This dataset comes with a label file where each row contains a time period where we know some anomalies occurred. This file contains 30 anomalous periods where the first and the second column respectively marks the start and the end of the periods.

The following are sample rows from this label dataset: **note the absence of any header or row number**.

![Synthetic pump data label overview](pictures/pump_label_data.png)

**Note:** all the timestamps from the training dataset and the labels should be within the same time zone.

![Signal overview](pictures/signal_overview.png)

### Alternative input format
In this tutorial, we will consider a single file as it is easier to manage at this stage.

However, your production system might provide sensor data in individual files (1 file per sensor or 1 file for a group of sensors). Having distinct files removes the need to align timestamps for every columns in your datafile and might be easier to process depending on how your data generation and collection pipeline look like.

Amazon Lookout for Equipment is flexible when it comes to ingesting your data and the service will adapt itself to these different situations as you will see at the ingestion step.

### Upload input data to S3
Navigate to the `label-data` folder in your S3 bucket and drag'n drop the `labels.csv` file there.

Then, navigate to the `facility` folder and upload your `pump_sensors.csv` file there: depending on your Internet connectivity to the selected AWS region, this upload can take a few minutes to complete:

![Dataset upload](pictures/dataset_upload.png)

**Congratulations, your S3 bucket is created and populated, you are ready to create your first Lookout for Equipment project!**

## Create a new Lookout for Equipment project
Now that your dataset is ready and uploaded to an S3 bucket, we are going to use the console to train a model and see the events detected in your dataset.

1. Access the Lookout for Equipment management console by typing **Lookout** in the search bar as shown below and click on Lookout for Equipment to open the service console:

![Lookout for Equipment console access](pictures/lookout_service_search.png)

2. Click on the **Create project** button and give a name to your project:

![Project creation](pictures/create_project.png)

3. After your project is created, you're brought to the project dashboard where you can click on **Add dataset** to ingest your time series data:

![Project dashboard](pictures/project_dashboard.png)

4. On the **Add dataset** screen:
    * Click on **Browse** and point to the S3 location where you uploaded your time series data
    * Then, select `Create a new role` for the **IAM role**: this lets Lookout for Equipment create a least privileged role that only has access to the S3 bucket selected in the previous step.
    * Then, select `By part of the filename` for the **Schema detection method**
    * Finally, choose `_` for the **Delimiter**

You should see screen similar toe the following one:

![Data ingestion](pictures/data_ingestion.png)

5. Click on the **Start ingestion** button. If you elected to let Lookout for Equipment to create a role on your behalf, a blue banner at the top will let you know the role is created and currently propagating through your account. A couple seconds later, the ingestion starts and you are brought back to the project dashboard where you can see the ingestion status:

![Data ingestion in progress](pictures/data_ingestion_progress.png)

6.  After 5 to 10 minutes, the ingestion is done. The banner at the top of the dashboard changes to green and displays a success message. During ingestion, Lookout for Equipment prepares the dataset so that it can be used for multiple algorithms at training time. The service also grades your signals and generates a data quality report. After ingestion, you can click on **View dataset** to visualize this report:

![Signal grading report](pictures/signal_grading_report.png)

This report grades each signal as:
* `High`: no significant validation errors were detected in the data during ingestion. Data from sensors in this category is considered the most reliable for model training and evaluation.
* `Medium`: one or more **potentially** harmful validation errors were detected in the data during ingestion. Data from sensors in this category is considered less reliable for model training and evaluation. The insights provided for the individual sensors may help you improve your dataset quality (for instance, you may have not noticed a lot of missing data on specific signals)
* `Low`: one or more significant validation errors were detected in the data during ingestion. There's a high probability that training a model on data from sensors in this category will result in poor model performance.

If you click on the individual sensor grades, you will see the reasons behind Lookout for Equipment grade. You can check out the [**documentation page**](https://docs.aws.amazon.com//lookout-for-equipment/latest/ug/reading-details-by-sensor.html?icmpid=docs_console_unmapped) dedicated to Evaluating sensor grades to know more about the error, explanations, actions taken by Lookout for Equipment on your behalf and actions recommended to you as a user.

Now that your time series dataset has been ingested, you can train an anomaly detection model.

## Training an anomaly detection model

We are going to create a new model from the **Dataset** page. To achieve this, you can follow these steps:

1. In the *Details by sensor* section, select all the sensors you want to use to train a model. You can select up to 300 fields to train a single model and you can use the table headers to sort your signals. Select all the fields available in this dataset by navigating the 2 pages available. Your screen should look similar to the following. Note the 30/30 at the top of this section which means 30 sensors out of the 30 availables have been selected:

![Training field selection](pictures/training_field_selection.png)

2. Click on **Create model**
3. Give a name to your model (`synthetic-pump-model` for instance) and click **Next**
4. Choose `2019-01-01` to `2019-07-31` for the **Training set date range** (the first 7 months: Lookout for Equipment needs a minimum of 90 days of data to train a model)
5. Choose `2019-08-01` to `2019-10-27` for the **Evaluation set date range** (the last 3 months)
6. For the **Sample rate** choose `15 minutes`
   * Sampling rate has an impact on the training time. The original sampling rate of this dataset is `1 minute`: leaving `No rate specified` means Lookout for Equipment will use this value, which will yields a training that lasts around 1.5 hour. For this workshop, you can start by downsampling you data to 15 minutes: training time will be around 15-20 minutes. Come back at this step and train a second model with a different sampling rate to appreciate the difference in the results for this particular dataset.
7. Leave the other fields as is and click **Next**
8. Browse to your S3 bucket and select the location where you upload the `labels.csv` file during the data preparation phase earlier.
   * **This is optional** and impacts heavily the training time.
   * You can launch a first training without this parameter and train another one afterward to visualize the difference.
9. On the **Review and train** section, scroll down and click on **Start training**. You are brought back to the project dashboard where you can see the training is now in progress:

![Training in progress](pictures/training_in_progress.png)

When your model has been trained, the model status transitions to *Completed*. Without labels, this training will take **~15 minutes** to train. If you decide to take the labels into account, the training time will be approximately **1.5 hour** with the default sampling rate of 1 minute and approximately **15-20 minutes** if you selected a downsampling to 15 minutes.

Now that your model has been trained, you can visualize the evaluation results for your model:

1. Click on **View models** from the project dashboard: the **Models** page is displayed where you can see status and creation time for each model trained in this project:

![Models list](pictures/models_list.png)

2. Click on the *Name* of the model you want to dive deeper on: the **Model details** page is shown. In the top section (*Model overview*), you will see the *Status* (`Ready for inference`) of your model and the *Training time* (`15 minutes`) that was needed to build it:

![Model overview](pictures/model_overview.png)

3. Scroll down to the *Model performance* section which will look similar to the following:

![Model performance](pictures/model_performance.png)

At the top, you get an overview of the performance of the model, telling you that it was able to:

* Capture 8 abnormal equipment behavior events in the label ranges (where known anomalies happen): Amazon Lookout for Equipment also computes an average forewarning time of 19 hours and 43 minutes for these events.
* Detect an addition 8 abnormal equipment behavior events outside of the label ranges with an average duration of 39 minutes.
* You also get a time range plot putting the labelled events (in blue) and the detected ones (in red) next to each other. You can use the slider to zoom and pan over the time range more precisely.

The *Event details* section unpacks up to the top 15 sensors for each detected events. When the page is displayed, the ranking shown are the one of the first detected event. You can select any detected event by clicking on one of the red bars above to update the *Event details* section. When you select a detected events, you will get a screen similar to the following:

![Top contributing sensors](pictures/model_top_sensors.png)

## Deploying a trained model

### Configuring an inference scheduler
Now that you have a trained model, you can deploy it by creating an inference scheduler. To do this, follow these steps:

1. On the model performance page, scroll down and select the **Inference schedule tab**. There, click on **Schedule inference**:

![Inference, initial page](pictures/inference_empty.png)

2. Enter a name for your scheduler: I called mine `synthetic-pump-model-scheduler`
3. Click on the **Browse** button next to the **Input data** S3 location and point to the **inference-data/input/** path in your bucket
4. Select a **Data upload frequency** of `15 minutes`: this parameter will have a minimum value equal to the resampling of your trained model.
5. Scroll down to the **Output data** section and select the **inference-data/output/** path in your bucket
6. Leave all the other parameters as is and click **Schedule inference** at the bottom of the screen.

After a few seconds, you scheduler is up and ready. It will wake up within the next 15 minutes and every 15 minutes after that. Once your scheduler is created, you are brought back to the model details page where you can see you model is ready for inference. If you click on the *Inference schedule* tab, you will see a screen similar to this one:

![Active scheduler](pictures/inference_active.png)

### Preparing inference data
Let’s prepare some data so that they are found by the scheduler: the input file must be a CSV file that follows the same structure than the original dataset. In this example, we had a single CSV file with one timestamp column followed by 30 columns, each containing an individual tag. [**Click here**](https://lookoutforequipmentbucket-us-east-1.s3.amazonaws.com/datasets/getting-started/console/pump_yyyymmddHHMM00.csv) to download a sample of this file.

![Synthetic pump data timeseries overview](pictures/pump_data_overview.png)

According to the `labels.csv` file, we know that an anomalous period was mentioned on **2019-10-17**. As an example, we are going to extract the *last 10 rows of that particular day* and put them in a new CSV file.

When Amazon Lookout for Equipment scheduler wakes up, given how we configured it in the previous step, it looks for a file with a name that will match this pattern: `{assetname}_{yyyyMMddHmmss}.csv`.

At the time this document was written, the date / time was **2022-07-29 at 11.36am UTC**: as the scheduler was configured to wake up every 15 minutes, we will save our CSV file with this name: `pump_20220729113000.csv` where `pump` is the name of our component in the original dataset schema. The scheduler will wake up at **11.45am UTC** (this is the next time that is a multiple of 15 minutes) and looks for a file generated 15 minutes before (11.45am minus 15 minutes is 11.30am). Using this naming convention will allow the scheduler to find this file and open it.

Next we need to make sure the timestamps *inside* the first column of this file also match the timestamp in the name: when the scheduler opens this file, it will indeed look for any row with a date that is between the inference execution start time minus 15 minutes and the actual execution time. In our case, we need to have at least one row with a timestamp between 11.30am UTC and 11.45pm UTC.

Update the timestamp column of your CSV file and make sure the UTC date and time matches the timestamp you used to name the file. Your initial file should look somewhat similar to this (note the timestamp column in UTC time *and* matching the timestamp in the filename):

![Inference input](pictures/inference_input.png)

Then, upload your inference data to your S3 bucket, in the `inference-data/input/` location. After a few minutes, your scheduler will wake up, look into the input location configured above, run the data found against your trained model and output the results in JSON format in the output location in Amazon S3. The scheduler *Inference history* is updated and should look similar to the following: if a file is not found, or if no suitable timestamp is found in the inference input, the status will be marked as *Failed*:

![Inference success](pictures/inference_success.png)

Navigate to your output Amazon S3 location and locate the JSON files to review its content and check the events potentially detected by Amazon Lookout for Equipment. For an example on how you can post-process and visualize these results, feel free to check out the *Python Notebook section* in the Getting Started folder of this GitHub repo. This is first JSON line you will see in this file if you open it:

```json
{
    "timestamp": "2022-07-29T11:45:00.000000", 
    "prediction": 1, 
    "prediction_reason": "ANOMALY_DETECTED", 
    "anomaly_score": 0.97399, 
    "diagnostics": [
        {"name": "pump\\Sensor0", "value": 0.04}, 
        {"name": "pump\\Sensor1", "value": 0.0}, 
        {"name": "pump\\Sensor10", "value": 0.06}, 
        {"name": "pump\\Sensor11", "value": 0.0}, 
        {"name": "pump\\Sensor12", "value": 0.06}, 
        
        [...]
        
        {"name": "pump\\Sensor9", "value": 0.0}
    ]
}

```

For each row with a timestamp that falls between two inference executions, you will get:

* The actual *timestamp*
* The *prediction* (0 for nothing detected and 1 if an anomaly is detected)
* The *raw anomaly score* (a float number between 0.0 and 1.0)
* If the prediction is 1, then you will also have a *diagnostics* field in the JSON file which will list each component\sensor pair and the contribution to the event as a percentage (similarly to what is output in the diagnostic page of a trained model)

## Conclusion
### Cleanup
Don't forget to stop your scheduler to stop incurring cost. If you stop the scheduler, your inference history will be maintained: if you delete the scheduler, you will lose the inference history.

You are free to leave the trained model and your project as they won't incur you anymore cost.

Otherwise, feel free to delete the model, then the project, empty your S3 bucket and delete it.